In [1]:
import pandas as pd
import numpy as np
import json, re

In [6]:
import requests, time
def refine_loc(x,y,name,deg_diff=1,fuzzy=.9,obj_id=None):
    url = "http://api.geonames.org/search"
    north = float(y)+deg_diff
    south = float(y)-deg_diff
    east = float(x)+deg_diff
    west = float(x)-deg_diff
    username = "jaguillette"
    Q = {'type':'json','username':username,'featureClass':'P','name':name,'north':north,'south':south,'east':east,'west':west,'fuzzy':fuzzy}
    R = requests.get(url,params=Q)
    georesponse = R.json()
    time.sleep(1)
    #print(georesponse)
    if obj_id:
        print("Object {}:".format(obj_id))
    if georesponse['totalResultsCount'] == 1:
        print("{} was found unambiguously.".format(georesponse['geonames'][0]['name']))
        #return georesponse['geonames'][0]['lng'], georesponse['geonames'][0]['lat']
        return georesponse['geonames'][0]['geonameId']
    elif georesponse['totalResultsCount'] > 1:
        print("{} returned ambiguous results. Using first result.".format(name))
        print(R.url)
        #return georesponse['geonames'][0]['lng'], georesponse['geonames'][0]['lat']
        return georesponse['geonames'][0]['geonameId']
    else:
        print("{} was not found. Returning None".format(name))
        print(R.url)
        #return x,y
        return None

In [3]:
refine_loc(36.672298,49.525859,'Андреевка')

Андреевка returned ambiguous results. Using first result.
http://api.geonames.org/search?name=%D0%90%D0%BD%D0%B4%D1%80%D0%B5%D0%B5%D0%B2%D0%BA%D0%B0&username=jaguillette&featureClass=P&south=48.525859&type=json&north=50.525859&west=35.672298&east=37.672298&fuzzy=0.9


713428

In [4]:
settlements = pd.DataFrame.from_csv("imperiia/settlementsCoordinates.csv",index_col=None)

In [23]:
#settlements.to_json("settlements.json")

In [5]:
settlements.head()

,wkt_geom,OBJECTID_1,Post,Category,Group,Placename,REGION,SHEET,UnitID,Status,Uz_name,Prov_name,XCOORD,YCOORD,EXCESS
0,POINT(49.67096033092822438 56.0384794262550372),1,0,selo or sloboda,rural,Andreevskoe Glukhovo tozhe,3,Atlas 041,59,Regular,Kazan,Kazanskaia,49.670960,56.038479,NaN
1,POINT(49.39587316522359117 55.9948585851506806),2,0,village (derevnia),rural,Saia Iskiiurt tozhe,3,Atlas 041,59,Regular,Kazan,Kazanskaia,49.395873,55.994859,NaN
2,POINT(49.41044635593399192 55.9521984363427265),3,0,selo or sloboda,rural,Kaimary,3,Atlas 041,59,Regular,Kazan,Kazanskaia,49.410446,55.952198,NaN
3,POINT(50.00489549548510126 55.93324127496659059),4,1,selo or sloboda,rural,Churilino,3,Atlas 041,59,Regular,Kazan,Kazanskaia,50.004895,55.933241,NaN
4,POINT(50.12309829443328368 55.97664168010962982),6,0,registered (pripisnoj) town,urban,Arsk,3,Atlas 041,59,Regular,Kazan,Kazanskaia,50.123098,55.976642,NaN


In [ ]:
settlements['geonamesId'] = settlements.apply(lambda row: refine_loc(row['XCOORD'],row['YCOORD'],row['Placename'],obj_id=row['OBJECTID_1']),axis=1)

Object 1:
Andreevskoe Glukhovo tozhe was not found. Returning None
http://api.geonames.org/search?name=Andreevskoe+Glukhovo+tozhe&username=jaguillette&featureClass=P&south=55.038479&type=json&north=57.038479&west=48.67096&east=50.67096&fuzzy=0.9
Object 2:
Saia Iskiiurt tozhe was not found. Returning None
http://api.geonames.org/search?name=Saia+Iskiiurt+tozhe&username=jaguillette&featureClass=P&south=54.994859&type=json&north=56.994859&west=48.395872999999995&east=50.395872999999995&fuzzy=0.9
Object 3:
Kaimary was not found. Returning None
http://api.geonames.org/search?name=Kaimary&username=jaguillette&featureClass=P&south=54.952197999999996&type=json&north=56.952197999999996&west=48.410446&east=50.410446&fuzzy=0.9
Object 4:
Churilino returned ambiguous results. Using first result.
http://api.geonames.org/search?name=Churilino&username=jaguillette&featureClass=P&south=54.933241&type=json&north=56.933241&west=49.004895&east=51.004895&fuzzy=0.9
Object 6:
Arsk was found unambiguously.
Ob

In [ ]:
settlements.head()

In [6]:
with open("resources/admindf.json","r",encoding="utf-8") as fp:
    admin = json.load(fp)

In [7]:
nfdf = pd.DataFrame.from_csv('output/no_geo.csv')

In [8]:
nfdf.head()

,ru_prov,ru_uezd,page,text,name,alt_name,_type,source_url,html_text,name_modern_sp,prov_modern_sp,uezd_modern_sp,prov_stem,uezd_stem,admin_hierarchy,partof_id,admin1_partof,admin2_partof
3,Рязанскаго Намѣстничества,Донковскомъ уѣздѣ,10,"АВДУЛОВО, ИВАНОВСКОЕ тожь, село Рязанскаго Нам...",Авдулово,Ивановское,село,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"АВДУЛОВО, ИВАНОВСКОЕ тожь, </span><span class=...",Авдулово,Рязанского Наместничества,Донковском уезде,рязанск,донковск,рязанск|донковск,62,28,62
4,Вологодскаго Намѣстничества,Вологодскаго уѣздѣ,10,"АВЖЕНСКАЯ , волость Вологодскаго Намѣстничеств...",Авженская,NaN,волость,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"АВЖЕНСКАЯ , </span><span class=""font0"" style=""...",Авженская,Вологодского Наместничества,Вологодского уезде,вологодск,вологодск,вологодск|вологодск,55,4,55
5,Костромскаго Намѣстничества,not_in_text,10,"АВРАМІЕВЪ, монастырь Костромскаго Намѣстничест...",Авраміевъ,NaN,монастырь,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"АВРАМІЕВЪ</span><span class=""font0"" style=""fon...",Аврамиев,Костромского Наместничества,not_иn_text,кострома,нот_ин_теxт,кострома,14,14,NaN
6,Нижегородскаго Намѣстничества,Арзамаскомъ уѣздѣ,10,"АДОВАТОВО, село Нижегородскаго Намѣстничества ...",Адоватово,NaN,село,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"АДОВАТОВО, </span><span class=""font0"" style=""f...",Адоватово,Нижегородского Наместничества,Арзамаском уезде,нижегородск,арзамаск,нижегородск|арзамаск,42,17,42
7,Тамбовскаго Намѣстничества,Темниковскомъ уѣздѣ,10,"АКСЕЛЬ, село Тамбовскаго Намѣстничества въ Тем...",Аксель,NaN,село,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"АКСЕЛЬ, </span><span class=""font0"" style=""font...",Аксель,Тамбовского Наместничества,Темниковском уезде,тамбовск,темниковск,тамбовск|темниковск,126,32,126


In [30]:
"ЙЬЪАЕИОУЭЮЯЁЫ"+"ЙЬЪАЕИОУЭЮЯЁЫ".lower()

'ЙЬЪАЕИОУЭЮЯЁЫйьъаеиоуэюяёы'

In [25]:
import re

In [39]:
lc_cyrillic_translit={u'\u0410': 'A', u'\u0430': 'a',u'\u0411': 'B', u'\u0431': 'b',u'\u0412': 'V', u'\u0432': 'v',u'\u0413': 'G', u'\u0433': 'g',u'\u0414': 'D', u'\u0434': 'd',u'\u0415': 'E', u'\u0435': 'e',u'\u0416': 'Zh', u'\u0436': 'zh',u'\u0417': 'Z', u'\u0437': 'z',u'\u0418': 'I', u'\u0438': 'i',u'\u0419': 'I', u'\u0439': 'i',u'\u041a': 'K', u'\u043a': 'k',u'\u041b': 'L', u'\u043b': 'l',u'\u041c': 'M', u'\u043c': 'm',u'\u041d': 'N', u'\u043d': 'n',u'\u041e': 'O', u'\u043e': 'o',u'\u041f': 'P', u'\u043f': 'p',u'\u0420': 'R', u'\u0440': 'r',u'\u0421': 'S', u'\u0441': 's',u'\u0422': 'T', u'\u0442': 't',u'\u0423': 'U', u'\u0443': 'u',u'\u0424': 'F', u'\u0444': 'f',u'\u0425': 'Kh', u'\u0445': 'kh',u'\u0426': 'Ts', u'\u0446': 'ts',u'\u0427': 'Ch', u'\u0447': 'ch',u'\u0428': 'Sh', u'\u0448': 'sh',u'\u0429': 'Shch', u'\u0449': 'shch',u'\u042a': '"', u'\u044a': '"',u'\u042b': 'Y', u'\u044b': 'y',u'\u042c': "'", u'\u044c': "'",u'\u042d': 'E', u'\u044d': 'e',u'\u042e': 'Iu', u'\u044e': 'iu',u'\u042f': 'Ia', u'\u044f': 'ia',u'\u0462': 'E', u'\u0463': 'e'}
bgn_translit = {"special_cases":[[r"([ЙЬЪАЕИОУЭЮЯЁЫйьъаеиоуэюяёы])е",r"\1ye"],[r"([ЙЬЪАЕИОУЭЮЯЁЫйьъаеиоуэюяёы])Е",r"\1Ye"],[r"([ЙЬЪАЕИОУЭЮЯЁЫйьъаеиоуэюяёы])ё",r"\1yë"],[r"([ЙЬЪАЕИОУЭЮЯЁЫйьъаеиоуэюяёы])ё",r"\1yë"],[r"([ЙЬЪАЕИОУЭЮЯЁЫйьъаеиоуэюяёы])Ё",r"\1Yë"],[r"^Е",r"Ye"],[r"^е",r"ye"],[r"^Ё",r"Yë"],[r"^ё",r"yë"]],u'\u0410': 'A', u'\u0430': 'a', u'\u0411': 'B', u'\u0431': 'b', u'\u0412': 'V', u'\u0432': 'v', u'\u0413': 'G', u'\u0433': 'g', u'\u0414': 'D', u'\u0434': 'd', u'\u0415': 'E', u'\u0435': 'e', u'\u0416': 'Zh', u'\u0436': 'zh', u'\u0417': 'Z', u'\u0437': 'z', u'\u0418': 'I', u'\u0438': 'i', u'\u0419': 'Y', u'\u0439': 'y', u'\u041a': 'K', u'\u043a': 'k', u'\u041b': 'L', u'\u043b': 'l', u'\u041c': 'M', u'\u043c': 'm', u'\u041d': 'N', u'\u043d': 'n', u'\u041e': 'O', u'\u043e': 'o', u'\u041f': 'P', u'\u043f': 'p', u'\u0420': 'R', u'\u0440': 'r', u'\u0421': 'S', u'\u0441': 's', u'\u0422': 'T', u'\u0442': 't', u'\u0423': 'U', u'\u0443': 'u', u'\u0424': 'F', u'\u0444': 'f', u'\u0425': 'Kh', u'\u0445': 'kh', u'\u0426': 'Ts', u'\u0446': 'ts', u'\u0427': 'Ch', u'\u0447': 'ch', u'\u0428': 'Sh', u'\u0448': 'sh', u'\u0429': 'Shch', u'\u0449': 'shch', u'\u042a': '"', u'\u044a': '"', u'\u042b': 'Y', u'\u044b': 'y', u'\u042c': "'", u'\u044c': "'", u'\u042d': 'E', u'\u044d': 'e', u'\u042e': 'Yu', u'\u044e': 'yu', u'\u042f': 'Ya', u'\u044f': 'ya'}

def transliterate(word, translit_table):
    """
    Transliterates 'word' based on the key/value pairs in 'translit_table'
    """
    if "special_cases" in translit_table.keys():
        for pair in translit_table['special_cases']:
            word = re.sub(pair[0],pair[1],word)
    converted_word = ''
    for char in word:
        transchar = ''
        if char in translit_table:
            transchar = translit_table[char]
        else:
            transchar = char
        converted_word += transchar
    return converted_word

In [10]:
nfdf['lc_translit'] = nfdf.name_modern_sp.apply(transliterate,args=[lc_cyrillic_translit])

In [11]:
matches = []
for n in nfdf.lc_translit:
    if n in settlements.Placename.tolist():
        matches.append(n)

In [15]:
settle_matches = settlements.ix[settlements.Placename.isin(matches)]
nfdf_matches = nfdf.ix[nfdf.lc_translit.isin(matches)]
merged = pd.merge(settle_matches,nfdf_matches,left_on='Placename',right_on='lc_translit')
merged.to_csv('imperiia/settle_merge.csv',encoding='utf-8')

In [13]:
import nltk

In [14]:
ru_stemmer = nltk.stem.SnowballStemmer('russian')

In [16]:
merged['translit_prov_stem'] = merged.prov_stem.apply(lambda text: transliterate(text,cyrillic_translit))

In [17]:
merged['settle_prov_stem'] = merged.Prov_name.apply(ru_stemmer.stem)
merged['settle_prov_stem'] = merged.settle_prov_stem.apply(ru_stemmer.stem).str.lower()

In [18]:
merged[['settle_prov_stem','translit_prov_stem']]

,settle_prov_stem,translit_prov_stem
0,orenburgsk,kavkazsk
1,orenburgsk,voronezhsk
2,viatsk,voronezhsk
3,orenburgsk,ufimsk
4,orenburgsk,tambovsk
5,arkhangel'sk,tambovsk
6,tobol'sk,tambovsk
7,orenburgsk,khar'kovsk
8,ekaterinoslavsk,khar'kovsk
9,ekaterinoslavsk,khar'kovsk


In [19]:
matches = merged.ix[merged.settle_prov_stem == merged.translit_prov_stem]
matches.to_csv('all_settle_matches.csv',encoding='utf-8')

In [43]:
matches[['Placename','Prov_name','Uz_name','lc_translit','ru_prov','ru_uezd']].head()

,Placename,Prov_name,Uz_name,lc_translit,ru_prov,ru_uezd
11,Andreevka,Khar'kovskaia,Zmiev (or Chuguev?),Andreevka,Харьковскаго Намѣстничества,Изюмскомъ уѣздѣ
23,Alekseevka,Voronezhskaia,Starobel'sk,Alekseevka,Воронежскомъ Намѣстничества,not_in_text
25,Troitsk,Penzenskaia,Krasnoslobodsk (Troitsk),Troitsk,Пензенскаго Намѣстничества,not_in_text
27,Shadrinsk,Permskaia,Shadrinsk,Shadrinsk,Пермскаго Намѣстничества,not_in_text
28,Irbit,Permskaia,Irbit,Irbit,Пермскаго Намѣстничества,not_in_text


In [21]:
matches[['name','alt_name','partof_id','XCOORD','YCOORD','_type','source_url','text']]

,name,alt_name,partof_id,XCOORD,YCOORD,_type,source_url,text
11,Андреевка,NaN,69,36.672298,49.525859,слобода,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"АНДРЕЕВКА, слобода Харьковскаго Намѣстничества..."
23,Алексѣевка,NaN,5,39.045304,49.015631,слобода,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"АЛЕКСѢЕВКА, слобода Графа Шереметева, въ Ворон..."
25,Троицкъ,NaN,22,43.677981,54.190731,родъ,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"ТРОИЦКЪ, съ 1780 году родъ Пензенскаго Намѣстн..."
27,Шадринскъ,NaN,23,63.593559,55.973286,городъ,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"ШАДРИНСКЪ, съ 1781 году городъ Пермскаго Намѣс..."
28,Ирбитъ,NaN,23,63.203104,57.528819,городъ,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"ИРБИТЪ, городъ Пермскаго Намѣстничества. Здѣсь..."
56,Покровское,NaN,115,37.387057,55.750661,село,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"ПОКРОВСКОЕ, село Московской Губерніи въ уѣздѣ ..."
60,Покровское,NaN,115,38.014248,55.622179,село,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"ПОКРОВСКОЕ, село Московской Губерніи въ уѣздѣ ..."
64,Покровское,NaN,115,37.042598,56.408362,село,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"ПОКРОВСКОЕ, село Московской Губерніи въ уѣздѣ ..."
97,Покровское,Любоша,87,38.978965,52.789522,село,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"ПОКРОВСКОЕ , ЛЮБОША тожь, село Орловскаго Намѣ..."
148,Троицкое,NaN,67,37.37474,55.421126,село,http://babel.hathitrust.org/cgi/pt?id=uc1.b458...,"ТРОИЦКОЕ, село Московской Губерніи въ Звенигор..."


In [48]:
import requests, time
def refine_loc(x,y,name):
    url = "http://api.geonames.org/search"
    north = float(y)+.6
    south = float(y)-.6
    east = float(x)+.6
    west = float(x)-.6
    name = transliterate(name,bgn_translit)
    Q = {'type':'json','username':'jaguillette','featureClass':'P','name':name,'north':north,'south':south,'east':east,'west':west}
    R = requests.get(url,params=Q)
    georesponse = R.json()
    time.sleep(1)
    try:
        print(georesponse['geonames'][0]['name'])
        return georesponse['geonames'][0]['lng'], georesponse['geonames'][0]['lat']
    except IndexError:
        print("{} was not found".format(name))
        print(R.url)
        return x, y

In [46]:
import numpy as np

In [50]:
matches['new_xcoord'], matches['new_ycoord'] = np.vectorize(refine_loc)(matches.XCOORD,matches.YCOORD,matches.name_modern_sp)

Andreyevka
Andreyevka
Alekseyevka
Troitsk
Shadrinsk
Irbit
Pokrovskoye
Pokrovskoye
Pokrovskoye
Pokrovskoye
Troitskoye
Troitskoye
Troitskoye
Troitskoye
Bredikhino
Rozhestvenskoye was not found
http://api.geonames.org/search?name=Rozhestvenskoye&username=jaguillette&featureClass=P&east=37.575171000000005&type=json&south=53.068498&west=36.375171&north=54.268498
Ivanovskoye
Ivanovskoye
Horodyshche
Horodyshche
Gorodishche
Gorodishche
Spaskoye was not found
http://api.geonames.org/search?name=Spaskoye&username=jaguillette&featureClass=P&east=41.053412&type=json&south=57.161175&west=39.853412&north=58.361175
Semënovskoye
Bilyarsk
Laishevo
Spask was not found
http://api.geonames.org/search?name=Spask&username=jaguillette&featureClass=P&east=43.613525&type=json&south=53.46635&west=42.413525&north=54.66635
Tsyvil'sk was not found
http://api.geonames.org/search?name=Tsyvil%27sk&username=jaguillette&featureClass=P&east=48.268387000000004&type=json&south=55.137351&west=47.068387&north=56.33735100000

In [52]:
matches.to_csv("imperiia/all_settle_matches_updated.csv",encoding='utf-8')

In [53]:
current_towns = pd.DataFrame.from_csv('output/hgr_test_set_5.csv')

In [56]:
matches[matches.text.isin(current_towns.description.tolist())==False].to_csv("imperiia/new_settle_matches_updated.csv",encoding='utf-8')

In [57]:
new_matches = pd.DataFrame.from_csv("imperiia/new_settle_matches_updated.csv")

In [74]:
new_data = new_matches[["name","alt_name","partof_id","new_xcoord","new_ycoord","_type","source_url","text","name_modern_sp","lc_translit","admin1_partof","admin2_partof"]]

In [94]:
def reverse_geocode(lng,lat):
    keepgoing = True
    while keepgoing == True:
        try:
            url = "https://maps.googleapis.com/maps/api/geocode/json"
            Q = {'latlng':"{},{}".format(lat,lng)}
            R = requests.get(url, params=Q)
            #print(R.url)
            geo_response = R.json()
            locality = 'error'
            admin1 = ""
            for component in geo_response['results'][0]['address_components']:
                if 'locality' in component['types']:
                    locality = component['long_name']
                elif 'administrative_area_level_1' in component['types']:
                    admin1 = component['long_name']
                elif 'country' in component['types']:
                    country = component['long_name']
                    countryCode = component['short_name']
            if locality == 'error':
                for component in geo_response['results'][0]['address_components']:
                    if 'administrative_area_level_2' in component['types']:
                        locality = component['long_name']
            print("\"{0}, {1}\",\"{2}\",\"{3}\"".format(locality,admin1,country,countryCode))
            time.sleep(1)
            keepgoing = False
        except IndexError:
            time.sleep(1)
            continue
        return "{}, {}".format(locality,admin1), country, countryCode

In [95]:
new_data['pres_loc'], new_data['pres_country'], new_data['country_code'] = np.vectorize(reverse_geocode)(new_data.new_xcoord,new_data.new_ycoord)

"Andriivka, Kharkiv Oblast","Ukraine","UA"
"Andriivka, Kharkiv Oblast","Ukraine","UA"
"Novoaidars'kyi district, Luhans'ka oblast","Ukraine","UA"
"Shadrinsk, Kurgan Oblast","Russia","RU"
"Pokrovskoye, Lipetsk Oblast","Russia","RU"
"Troitskoye, Moscow Oblast","Russia","RU"
"Dubensky District, Tula Oblast","Russia","RU"
"Plavsk, Tula Oblast","Russia","RU"
"Ivanovskoye, Moscow Oblast","Russia","RU"
"Nekrasovsky District, Yaroslavl Oblast","Russia","RU"
"Semenovskoye, Yaroslavl Oblast","Russia","RU"
"Zubovo-Polyansky District, Mordovia","Russia","RU"
"Tsivilskiy rayon, Chuvashia Republic","Russia","RU"
"error, Kharkiv Oblast","Ukraine","UA"
"Bely, Tver Oblast","Russia","RU"
"Kolomats'kyi district, Kharkiv Oblast","Ukraine","UA"
"Orikhivs'kyi district, Zaporiz'ka oblast","Ukraine","UA"
"Novopetrivka, Zaporiz'ka oblast","Ukraine","UA"
"Pavlohrad, Dnipropetrovsk Oblast","Ukraine","UA"
"Yampil', Sums'ka oblast","Ukraine","UA"
"Svativs'kyi district, Luhans'ka oblast","Ukraine","UA"
"Dvorichans'k

In [78]:
dataset = pd.DataFrame.from_csv("output/hgr_test_set_5.csv")

In [79]:
partof_prov_lookup_dict = dataset[dataset.partof_prov == dataset.index][['partof_prov_name']].to_dict()
partof_uezd_lookup_dict = dataset[dataset.partof_uezd == dataset.index][['partof_uezd_name']].to_dict()

In [80]:
new_data['partof_prov_name'] = new_data.admin1_partof.apply(lambda x: partof_prov_lookup_dict['partof_prov_name'][x])

In [83]:
def uezd_lookup(i):
    if not np.isnan(i):
        return partof_uezd_lookup_dict['partof_uezd_name'][i]
    else:
        return None

In [85]:
new_data.admin2_partof = new_data.admin2_partof.replace(102.0,101.0)
new_data['partof_uezd_name'] = new_data.admin2_partof.apply(uezd_lookup)

c:\Anaconda3\lib\site-packages\pandas\core\generic.py:1858: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
  self[name] = value


In [86]:
new_data.to_csv("imperiia/matches_clean.csv",encoding='utf-8')

In [90]:
new_data.columns

Index(['name', 'alt_name', 'partof_id', 'new_xcoord', 'new_ycoord', '_type', 'source_url', 'text', 'name_modern_sp', 'lc_translit', 'admin1_partof', 'admin2_partof', 'partof_prov_name', 'partof_uezd_name'], dtype='object')

Working with admin dict creation
================================
This should go in its own notebook, but not today.

In [78]:
admdf = pd.DataFrame.from_dict(admin)
admdf['name']=admdf.index
admdf

,prov_stem,uezd_stem,name
алатырьск,NaN,40,алатырьск
алексин,NaN,41,алексин
алексинск,NaN,41,алексинск
арзамаск,NaN,42,арзамаск
арсамазск,NaN,42,арсамазск
архангельск,2,2,архангельск
ахтырк,NaN,43,ахтырк
ахтырск,NaN,43,ахтырск
бабиновецк,NaN,44,бабиновецк
бежецк,NaN,45,бежецк


In [23]:
import nltk
ru_stemmer = nltk.stem.SnowballStemmer('russian')

import re,json
from lxml import html

lookup_dict = {}
with open("special_cases.json",'r') as fp:
    lookup_dict = json.load(fp)

def ortho_rules(text,lookup_dict):
    """
    Takes a text string and a lookup dict as input
    Removes any lingering html tags
    Breaks the string down into component words with split
    Uses replacements to modernize orthography
    Returns re-assembled text with modern orthography
    """
    if text is not None:
        text = html.fromstring(text).text_content()
        textlist = text.split(' ')
        out_list = []
        for word in textlist:
            if word in [',','.',';',':']:
                out_list.append(word)
            word = word.strip(',.;:')
            if word in lookup_dict:
                word = lookup_dict[word]
                out_list.append(word)
            else:
                word = re.sub("[Ъъ]$",'',word)
                word = re.sub(r"^([че]{0,2}[ч]?[вбнр]?[веоаи])з([ПФКТШСЧпфктшсч])",r"\1с\2",word,re.IGNORECASE)
                word = re.sub("аго$","ого",word,re.IGNORECASE)
                word = re.sub("яго$","его",word,re.IGNORECASE)
                word = re.sub("ыя$","ые",word,re.IGNORECASE)
                word = re.sub("iя$","ие",word,re.IGNORECASE)
                word = word.replace('i','и') #english
                word = word.replace('I','И') #english
                word = word.replace('і','и') #cyrillic
                word = word.replace('І','И') #cyrillic
                word = word.replace('Ѣ','Е')
                word = word.replace('ѣ','е')
                out_list.append(word)
        out_str = ' '.join(out_list)
        return out_str
    else:
        return "None"

In [26]:
admdf['name_stem'] = admdf.name.apply(ortho_rules,args=[lookup_dict]).apply(ru_stemmer.stem)

In [31]:
admdf.to_json('admindf.json')

In [33]:
admdf.pivot_table(index='name_stem').to_json('admindf.json')

In [34]:
with open('admindf.json','r') as fp:
    admin_lookup = json.load(fp)

In [35]:
admin_lookup['uezd_stem']['донковск']

62.0